In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from ODE.latentSearchBatch import NormalLatentSearch
from glob import glob
import os

# Specify generator:
The function **generator_function** takes in input a set of latent vectors $z$ with shape $[n, m]$ and outputs a set of images with shape $[n, s, s, 3]$.

In [ ]:
# Path of the tensorflow_hub module (i.e., the pre-trained generator)
model_path = 'MODEL_DIR/SVHN_DCGAN_normal_latent_512/'

print("{} generator loading...".format(model_path))
generator_module = hub.Module(model_path)

# Wrap here your generator:
def generator_function(z):
    return generator_module(z, signature='G')

# Setup 

In [ ]:
# list of target instances
target_paths = glob('TARGET_IMAGES/**/*.png')
# dimension of the generator latent space
zsize = 512
# shape of the generator output
xshape = (32, 32, 3)
# used distance function over the data space
distance_function = NormalLatentSearch.MSE

# optional
optimizer = tf.train.AdamOptimizer
learning_rate = 0.01
# weights for each moment penalty term
moments_penalty = 0.5, 0.5, 0.5, 0.1, 0.5, 0.5


ls = NormalLatentSearch(
        xtarget_paths=target_paths,
        zsize=zsize,
        xshape=xshape,
        optimizer=optimizer,
        lr=learning_rate,
        distance_function=distance_function,
        moments_penalty=moments_penalty
)

# Run latent search

In [ ]:
# number of training iterations
iterations_number = 30000
# number of tensorboard logging operation during the training (set o for no logging)
logs_num = 100
# tensorboard logging base directory
log_path = './LOGS'

xhat_group_val, xtarget_group_val = ls(generator_function, iterations_number, logs_num, log_path)